In [1]:
# %matplotlib ipympl
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *

In [7]:
fig = plt.figure(figsize=(8, 6))
fig.show()

In [13]:
data = []

config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_values": [False, False, True, False],
#     "plot_pins": [False, False, False, False],
    "gpio_delay_time" : 0.010795,
    "fig": fig,
    "verbose": 0,
}

fig.clf()

with DGILibExtra(**config_dict) as dgilib:
    dgilib.device_reset()
    # data = dgilib.logger.update_callback()
    data = dgilib.logger.log(1)
#     dgilib.ax.plot(*dgilib.power_filter_by_pin(2))
#     dgilib.ax.plot(*dgilib.power_filter_by_pin(3))

Interfaces:
	 256:   power,   samples:    2000
	  48:   gpio,    samples:      42

Interfaces:
	 256:   power,   samples:    2000
	  48:   gpio,    samples:      82

Interfaces:
	 256:   power,   samples:    6000
	  48:   gpio,    samples:      82

Interfaces:
	 256:   power,   samples:    6000
	  48:   gpio,    samples:     146

Interfaces:
	 256:   power,   samples:   11000
	  48:   gpio,    samples:     146

Interfaces:
	 256:   power,   samples:   11000
	  48:   gpio,    samples:     164

Interfaces:
	 256:   power,   samples:   16000
	  48:   gpio,    samples:     164

Interfaces:
	 256:   power,   samples:   18000
	  48:   gpio,    samples:     164



In [6]:
print(data)

Interfaces:
	 256:   power,   samples:   15000
	  48:   gpio,    samples:     164



D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\widgets.py:346: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0.0
  ax.set_xlim((valmin, valmax))
D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\axes\_base.py:1717: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0.0
  self.set_xlim([v[0], v[1]], emit=emit, auto=False)
